# Homework 6

In [ ]:
# Setup matplotlib animation
import matplotlib
matplotlib.rc('animation', html='jshtml')

## Imports and Utilities
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:
from typing import Callable, Sequence, List

from collections import namedtuple
import numpy as np
import dataclasses


@dataclasses.dataclass(frozen=True)
class Gaussian:
    mean: float  # Mean of a Gaussian distribution
    var: float  # Variance of a Gaussian distribution


class Simulator_1d:
    """A simulator for a robot moving in 1D.

    The simulator simulates the motion and sensor noises of the robot by sampling from 1D Gaussians.
    """

    def __init__(self,
                 init_state: float = 0.0,
                 process_var: float = 0.0,
                 measurement_var: float = 0.0,
                 seed: int = 0):
        # Initial simulator state
        self.init_state = init_state
        self.process_std = np.sqrt(process_var)
        self.measurement_std = np.sqrt(measurement_var)
        self.rng = np.random.default_rng(seed)

        self.state = self.init_state

    def simulate_motion(self, u: float):
        """Simulate robot motion. This function updates the value of
        self.state.
        """
        motion_noise = self.rng.normal(0, self.process_std)
        self.state = self.state + u + motion_noise

    def simulate_sensing(self) -> float:
        """Simulate the robot sensing process. Returns a measurement of the robot's current location.
            Measurement is based on the current location, with a Gaussian noise added.

        Returns:
            measurement: a scalar representing the measurement of robot's location.
        """
        sensor_noise = self.rng.normal(0, self.measurement_std)
        return self.state + sensor_noise

    def simulate_step(self, control: float, return_gt_loc: bool = False):
        """
            Run the simulation for one step with given control input, and returns the sensing result.
        """
        self.simulate_motion(control)
        if return_gt_loc:
            return self.simulate_sensing(), self.state
        return self.simulate_sensing()





## Kalman Filter Warmup


### Question
Return an `Gaussian` object with mean=0, std=2. 
    Please see the provided code for the definition of `Gaussian`.

For reference, our solution is **1** line(s) of code.

In [ ]:
def kf_warmup() -> Gaussian:
    """Return an `Gaussian` object with mean=0, std=2.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def kf_warmup_test():
    dist = kf_warmup()
    assert dist.mean == 0
    assert dist.var == 4

kf_warmup_test()

print('Tests passed.')

## Kalman filter process model


### Question
Implement the process step of a 1D Kalman filter, given a Gaussian prior distribution, 
  a process model $a$, $\sigma^2_w$, and a control $u$.

For reference, our solution is **4** line(s) of code.

In [ ]:
def kf_process(state_prior: Gaussian, a: float, square_sigma_w: float,
               u: float) -> Gaussian:
    """Implement the process step of a 1D Kalman filter. 
    Given a Gaussian prior distribution, a process model $a$, $\sigma^2_w$, and a control $u$.

    Returns: 
        an updated Gaussian.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def kf_process_test1():
    state_prior = Gaussian(4, 4)
    pred_state = kf_process(state_prior, 2, 2, 0)
    assert np.allclose([pred_state.mean, pred_state.var], [8, 18], atol=1e-4)

kf_process_test1()


def kf_process_test2():
    state_prior = Gaussian(4, 1)
    pred_state = kf_process(state_prior, 1, 1, 1)
    assert np.allclose([pred_state.mean, pred_state.var], [5, 2], atol=1e-4)

kf_process_test2()

print('Tests passed.')

## Kalman filter measurement model


### Question
Implement the measurement step of a 1D Kalman filter, given a prior mean and variance, 
  a measurement model $h$, $\sigma^2_v$, and a received measurement $y$.  

For reference, our solution is **18** line(s) of code.

In [ ]:
def kf_measurement(pred_state: Gaussian, h: float, square_sigma_v: float,
                   y: float) -> Gaussian:
    """Implement the measurement step of a 1D Kalman filter, given a prior mean and variance,
        a measurement model $h$, $\sigma^2_v$, and a received measurement $y$.

    Returns:
        an updated Gaussian given the measurement.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def kf_measurement_test1():
    state = Gaussian(5, 2)
    state_posterior = kf_measurement(state, 1, 1, 5)
    assert np.allclose([state_posterior.mean, state_posterior.var],
                       [5., 0.6667],
                       atol=1e-4)

kf_measurement_test1()


def kf_measurement_test2():
    state = Gaussian(5, 2)
    state_posterior = kf_measurement(state, 1, 1, 10)
    assert np.allclose([state_posterior.mean, state_posterior.var],
                       [8.3334, 0.6667],
                       atol=1e-4)

kf_measurement_test2()

print('Tests passed.')

## Kalman filter


### Question
Combine the process and measurement steps into a single estimator that takes a prior,
a process model, a measurement model, a list of controls, measurements, and returns a posterior.

For reference, our solution is **9** line(s) of code.

In addition to all the utilities defined at the top of the Colab notebook, the following functions are available in this question environment: `kf_measurement`, `kf_process`. You may not need to use all of them.

In [ ]:
def kf(state_prior: Gaussian, a: float, square_sigma_w: float, h: float,
       square_sigma_v: float, controls: Sequence[float],
       measurements: Sequence[float]) -> Gaussian:
    """Combine the process and measurement steps into a single estimator that takes a prior,
        a process model, a measurement model, a list of controls, measurements, and returns a posterior.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def kf_test1():
    state_init = Gaussian(0, 40)
    a = 1.
    h = 1.
    square_sigma_w = 1.
    square_sigma_v = 2.

    simulator = Simulator_1d(process_var=square_sigma_w,
                             measurement_var=square_sigma_v)
    controls = [1.0 for _ in range(10)]
    measurements = [simulator.simulate_step(u) for u in controls]

    state = kf(state_init, a, square_sigma_w, h, square_sigma_v, controls,
               measurements)
    assert np.allclose([state.mean, state.var], [7.2195, 1.000], atol=1e-4)

kf_test1()


def kf_test2():
    state_init = Gaussian(0, 400)
    a = 1.
    h = 1.
    square_sigma_w = 4.
    square_sigma_v = 1.

    simulator = Simulator_1d(process_var=square_sigma_w,
                             measurement_var=square_sigma_v)
    controls = [2. for _ in range(40)]
    measurements = [simulator.simulate_step(u) for u in controls]

    state = kf(state_init, a, square_sigma_w, h, square_sigma_v, controls,
               measurements)
    assert np.allclose([state.mean, state.var], [87.0068, 0.8284], atol=1e-4)

kf_test2()

print('Tests passed.')

## Kalman Filter Visualization


### Question
We have created a visualization tool in Colab for the filtering process. 
    You can run the code blocks and an animation of the filtered Gaussian in each step will be shown.
    There is also a plot showing the line chart of ground-truth locations, measurements, and posteriors.

In [ ]:
import matplotlib.pyplot as plt


def plot_gaussian(dist: Gaussian, label: str, ax=None):
    mean, var = dist.mean, dist.var

    xmin, xmax, ymin, ymax = -5, 15, 0, 0.6
    x = np.linspace(xmin, xmax, 100)
    y = np.exp(-0.5 * np.square(x - mean) / var) / np.sqrt(2 * np.pi * var)
    if ax is None:
        fig, ax = plt.subplots()

    ax.set_xlim([xmin, xmax])
    ax.set_ylim([ymin, ymax])

    ax.set_title(label)
    ax.plot(x, y)


PlotData = namedtuple("PlotData", ["name", "data"])


def render_animation(data_list: List[PlotData], plot_fn: Callable):
    """Render simulation as a matplotlib animation."""
    import matplotlib.animation as animation
    from IPython import display as display

    fig, ax = plt.subplots()

    def render_frame(i):
        ax.clear()
        plot_fn(*data_list[i].data, label=data_list[i].name, ax=ax)
        return ax

    anim = animation.FuncAnimation(fig,
                                   render_frame,
                                   len(data_list),
                                   interval=1000)
    video = anim.to_jshtml()
    html = display.HTML(video)
    display.display(html)
    plt.close()


def plot_time(gt_locs,
              measurements,
              posterior,
              label='Kalman Filtering',
              ax=None):
    xmin, xmax, ymin, ymax = 0, 10, -5, 15
    x = range(xmin, xmax + 1)

    if ax is None:
        fig, ax = plt.subplots()

    ax.set_xlim([xmin, xmax])
    ax.set_ylim([ymin, ymax])

    ax.set_title(label)

    ax.plot(x, gt_locs, label='gt_loc')
    ax.plot(x,
            measurements,
            label='measurements',
            marker='o',
            markerfacecolor='none')

    all_mu, all_var = map(np.asarray,
                          zip(*[[pos.mean, pos.var] for pos in posterior]))
    all_std = np.sqrt(all_var)
    std_top = all_mu + all_std
    std_btm = all_mu - all_std
    ax.plot(x, all_mu, label='filter')
    ax.plot(x, std_top, linestyle='--', color='g')
    ax.plot(x, std_btm, linestyle='--', color='g')
    ax.fill_between(x, std_btm, std_top, facecolor='g', alpha=0.1)

    legend = plt.legend(loc='upper right')
    plt.show()

This code visualizes the first test example (kf_test1). 
    You can take a look on the animation. 
    Please note the changing in variance of process step and measurement step.

In [ ]:
def visualize_kalman_test1():
    state_init = Gaussian(0, 40)
    a = 1.
    h = 1.
    square_sigma_w = 1.
    square_sigma_v = 2.
    num_step = 10

    simulator = Simulator_1d(process_var=square_sigma_w,
                             measurement_var=square_sigma_v)
    controls = [1.0 for _ in range(num_step)]
    measurements, gt_locs = map(
        list,
        zip(*[simulator.simulate_step(u, return_gt_loc=True)
              for u in controls]))

    result_list = [PlotData('init', [state_init])]

    state = state_init
    for i in range(num_step):
        result_list.append(
            PlotData(f'step_{i:02d}_process',
                     [kf_process(state, a, square_sigma_w, controls[i])]))
        state = kf(state, a, square_sigma_w, h, square_sigma_v, [controls[i]],
                   [measurements[i]])
        result_list.append(PlotData(f'step_{i:02d}_posterior', [state]))

    gt_locs.insert(0, 0)
    measurements.insert(0, 0)
    render_animation(result_list, plot_gaussian)
    plot_time(gt_locs, measurements, [
        res.data[0]
        for res in result_list
        if 'posterior' in res.name or 'init' in res.name
    ])


visualize_kalman_test1()